In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
from tokenize_function import tokenize

In [3]:
NnL = pd.read_excel('1.Dataset_files/CharacterData.xlsx',index_col=0)

In [4]:
edge_aliases = {0: 'Mr. Dursley', 1: 'Mrs. Dursley', 3: 'Mrs. Potter', 5: 'Mr. Potter', 7: 'My Lord',
                11: 'Dumbledore', 12: 'McGonagall', 13: 'Diggle', 14: 'Pomfrey',
                15: 'Hagrid', 18: 'Mrs. Figg', 21: 'Mr. Paws', 28: 'Boa Constrictor',
                37: 'Mr. Evans', 38: 'Mrs. Evans', 53: 'Quirrell', 57: 'Malfoy',
                62: 'Mr. Ollivander', 65: 'Mrs. Weasley', 69: 'Mr. Weasley',
                70: 'Longbottom', 78: 'Agrippa', 79: 'Ptolemy', 80: 'Grindelwald',
                90: 'Miss Granger', 91: 'Crabbe', 92: 'Goyle', 95: 'Friar',
                116: 'Nearly Headless Nick', 118: 'Mr. Finnigan', 119: 'Mrs. Finnigan', 
                121: 'Snape', 122: 'Filch', 123: 'Madam Hooch', 125: 'Mrs. Norris', 
                126: 'Professor Sprout', 127: 'Professor Binns', 130: 'Professor Flitwick', 
                132: 'Wood', 139: 'Flint', 140: 'Spinnet', 143: 'Pucey', 
                144: 'Higgs', 145: 'Madam Pince', 153: 'giant squid', 155: 'Mr. Mason',
                156: 'Mrs. Mason', 164: 'Lockhart', 171: 'Mr. Borgin', 173: 'Mr. Granger',
                174: 'Mrs. Granger', 175: 'Dr. Filibuster', 177: 'Manager of Flourish and Blotts',
                181: 'Mr. Creevey', 187: 'Sir Patrick', 189: 'D. J. Prod', 195: 'Mrs. Skower',
                204: 'Macmillan', 206: 'Professor Sinistra', 211: 'Dippet', 213: 'governors', 217: 'Tom',
                234: 'Lupin', 235: 'Professor Kettleburn', 238: 'Professor Trelawney',
                254: 'Professor Vector', 256: 'Davies', 257: 'Warrington', 258: 'Montague',
                259: 'Derrick', 260: 'Bole', 262: 'Macnair', 266: 'Mrs. Riddle', 267: 'Mr. Riddle',
                284: 'Krum', 285: 'Mr. Diggory', 289: 'Mr. Roberts', 290: 'Mr. Payne', 
                296: 'Mr. Wood', 297: 'Mrs. Wood', 301: 'Bode', 302: 'Croaker', 303: 'Pontner',
                312: 'Dimitrov', 313: 'Ivanova', 314: 'Zograf', 315: 'Levski', 316: 'Vulchanov',
                317: 'Volkov', 319: 'Ryan', 324: 'Lynch', 325: 'Mostafa', 326: 'Mrs. Roberts',
                329: 'Madam Maxime', 332: 'Skeeter', 333: 'Mad-Eye', 346: 'Master Barty',
                350: 'Karkaroff', 356: 'Gregorovitch', 369: 'Professor Grubbly-Plank', 381: 'Rosier',
                386: 'Mrs. Fudge', 388: 'Dolohov', 390: 'Rookwood', 395: "Frank Longbottom's wife",
                399: 'Mrs. Diggory', 400: 'Mr. Krum', 401: 'Mrs. Krum', 418: 'Mr. Prentice',
                420: 'Shacklebolt', 423: 'Podmore', 447: 'Scrimgeour', 451: 'Umbridge',
                452: 'Slinkhard', 464: 'Mr. Lovegood', 478: 'Mrs. Chang', 489: 'Barnabas the Barmy', 490: 'Mr. Edgecombe',
                491: 'Mr. Chang', 496: 'Kirke', 497: 'Slope', 501: 'Fortescu',
                506: 'Healer Smethwyck', 507: 'Pye', 511: 'Healer Strout', 518: 'Dawlish',
                519: 'Mr. Montague', 520: 'Mrs. Montague', 526: 'Carmichael', 527: 'Dingle',
                539: 'Dr. Ubbly', 541: 'Mrs. Goyle', 548: 'Yaxley', 549: 'Carrows', 550: 'Carrows',
                551: 'Greyback', 552: 'Slughorn', 562: 'McLaggen', 563: 'Belby',
                565: 'Mr. Belby', 575: 'Ogden', 576: 'Mr. Gaunt', 579: 'Mr. Patil',
                580: 'Mrs. Patil', 581: 'Mr. Midgen', 582: 'Mrs. Abbott', 583: 'Robins',
                584: 'Peakes', 585: 'Coote', 586: 'Bobbin', 588: 'Burkes',
                589: 'Mrs. Cole', 590: 'Stubbs', 591: 'Whalley', 592: 'Benson', 593: 'Bishop',
                603: 'Professor Merrythought', 607: 'Twycross', 622: 'Bungs', 626: 'Thicknesse',
                627: 'Professor Burbage', 627: 'Gorgovitch', 652: 'Cattermole', 653: 'Runcorn',
                669: 'youngest brother', 680: 'oldest brother'}

In [5]:
texts = np.zeros(len(NnL), dtype=object)
tokens = np.zeros(len(NnL), dtype=object)
aliases = np.zeros(len(NnL), dtype=object)

for char_idx in tqdm(range(len(NnL))):
    query = requests.get(f"https://harrypotter.fandom.com/api.php?action=parse&page={NnL['Link'].iloc[char_idx].split('/')[-1].split('#')[0]}&format=json").json()
    HTML = query['parse']['text']['*']
    soup = BeautifulSoup(HTML, 'html.parser')

    # Get aliases
    try:
        # Character has aliases
        alias_ = soup.find(string='Also known as').findNext("div")
        if alias_.find_all('li') != []:
            # Character has multiple aliases
            alias_ = [i.get_text() for i in alias_.find_all('li')]
            alias_ = [re.split('\s?(\()|(\[)',j)[0] for j in alias_]
        else:
            # Character has only one alias
            alias_ = [re.split('\s?(\()|(\[)',alias_.get_text())[0]]
    except:
        # Character has no aliases
        alias_ = []

    # Ensure that aliases are not the same as a character name
    alias_ = [i for i in alias_ if not NnL['Name'].str.contains(f'{i}', regex=False).any()]

    # Add edge cases
    if char_idx in edge_aliases:
        alias_.append(edge_aliases[char_idx])
    
    # Add first name to aliases
    none_aliases = ['A', 'Mr', 'Mrs', 'Dr', 'Manager', 'The', 'Father','Sorting', 'wizard', 'Chancellor', 'Workmen', 'Waitress', 'Sir',
                    'Nearly-Headless', 'Fat','Aged', 'Blood-Sucking', 'Forbidden','Unidentified', 'Zoo', 'Kepper', 'Muggle', 'Muggle-Born',
                    'Senior', 'Junior', 'Board', 'Committee','Academy', 'Ministry', 'Department',
                    'Little', 'Great', 'Old', 'Young', 'Head', 'Headmaster', 'Headmistress','Weird','Care', 'Montgomery', 'Hogwarts', 'Frank']
    if ((NnL['Name'].iloc[char_idx].split(' ')[0] not in none_aliases) and
    ("'s" not in NnL['Name'].iloc[char_idx]) and 
    ("s'" not in NnL['Name'].iloc[char_idx]) and
    ("family" not in NnL['Name'].iloc[char_idx]) and
    (NnL['Name'].iloc[char_idx].split(' ')[0].isalpha())):
        alias_.append(NnL['Name'].iloc[char_idx].split(' ')[0])

    # Ensure that aliases are unique and not the same as a character name
    alias_ = list(set(alias_))
    alias_ = [i for i in alias_ if i != NnL['Name'].iloc[char_idx]]

    # Save aliases
    aliases[char_idx] = alias_

    temp = [i.get_text() for i in soup.find_all('p', class_ = None)]

    # Filter out unwanted text
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'The title of this article is conjectural.' in temp[0]:
        temp = temp[1:]
    if 'This entry needs to be cleaned up to conform to a higher standard of quality.' in temp[0]:
        temp = temp[1:]
    if 'The topic of this article is of a real-life subject' in temp[0]:
        temp = temp[1:]
    if 'The title of this article intentionally uses incorrect spelling or grammar,' in temp[0]:
        temp = temp[1:]
    if 'This page "shows not" an image, but just a written article.' in temp[0]:
        temp = temp[1:]
    if 'Point me!' in temp[0]:
        temp = temp[1:]
    if 'Point me!' in temp[0]:
        temp = temp[1:]
    if 'At least some content in this article' in temp[0]:
        temp = temp[1:]
    if 'The title of this article is conjectural.' in temp[0]:
        temp = temp[1:]
    if 'Biographical information' in temp[0]:
        temp = temp[1:]

    # Get text    
    texts[char_idx] = ' '.join(temp)
    tokens[char_idx] = tokenize(texts[char_idx])

# Save
NnL['WikiText'] = texts
NnL['Tokens'] = tokens
NnL['Aliases'] = aliases

100%|██████████| 707/707 [09:02<00:00,  1.30it/s]


In [10]:
NnL

,Name,Link,WikiText,Tokens,Aliases
0,Vernon Dursley,/wiki/Vernon_Dursley,"Vernon Dursley was an English Muggle, husband ...","[vernon, dursley, english, muggle, husband, pe...","[Mr. Dursley, Vernon]"
1,Petunia Dursley,/wiki/Petunia_Dursley,Petunia Dursley (née Evans) (pre 1960 – pre 20...,"[petunia, dursley, née, evans, pre, pre, engli...","[Tuney, Petunia, Mrs. Dursley]"
2,Dudley Dursley,/wiki/Dudley_Dursley,Dudley Dursley (b. 23 June 1980)[1] was the Mu...,"[dudley, dursley, b, june, muggle, son, vernon...","[Diddy, Diddykins, Dudders, Dudley, Popkin, My..."
3,Lily Potter,/wiki/Lily_J._Potter,Lily J.[8] Potter (née Evans) (30 January[1] 1...,"[lily, j, potter, née, evans, january, october...","[Lily, Mrs. Potter]"
4,James Potter,/wiki/James_Potter_I,"England,[2] Great Britain Godric's Hollow, Wes...","[england, great, britain, godric, hollow, west...","[Prongs, James]"
...,...,...,...,...,...
702,Hugo Weasley,/wiki/Hugo_Granger-Weasley,Hugo Granger-Weasley was born around 2008 to a...,"[hugo, born, around, father, mother, making, t...",[Hugo]
703,Scorpius Malfoy,/wiki/Scorpius_Malfoy,Scorpius Hyperion Malfoy (b. c. 2006) was a Br...,"[scorpius, hyperion, malfoy, british, wizard, ...","[Scorpius, Scorpius the Dreadless, Scorpion Ki..."
704,Victoire Weasley,/wiki/Victoire_Weasley,Victoire Weasley (b. 2 May[1] 2000 or 2001)[2]...,"[victoire, weasley, b, may, witch, eldest, chi...",[Victoire]
705,Astoria Greengrass,/wiki/Astoria_Malfoy,Great Britain Astoria Malfoy[8] (née Greengras...,"[great, britain, astoria, malfoy, née, greengr...",[Astoria]


In [13]:
NnL.to_csv(r'1.Dataset_files/CharacterWikis.csv',index=False)